Recall that in class we have looked the gym model:
- The students arrive at the gym sequentially. We assume an exponential interarrival time with a mean of 2 minutes (e.g., an average of 30 students arriving per hour).
- Students work out for a random amount of time (uniform between 1 and 20).  20% of the students leave after working out. 80% go to the locker room (e.g., to shower and change) for a random amount of time (normally distributed with a mean of 20 and standard deviation of 5). Then these students leave the gym.

Here is the implementaion using python `class`.

## 1. Sample codes from class

In [1]:
!pip install simpy
import simpy
import random
import numpy as np
from functools import *
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
class simShell:
  def __init__(self,env,data):
    '''Generic sim code for single arrival process with sequential arrivals'''
    self.env = env                            # Define env as attribute of class 'simShell' (included as an argument to class methods via 'self')
    self.data = data                          # Define data as attribute of class 'simShell' to collect simulation data
    self.env.process(self.arrival())          # Begin arrival process
  def arrival(self):
    i = 0
    while True:
      i = i + 1                               # Arrival number
      time = np.random.exponential(3)         # Define interarrival time for entity i
      yield self.env.timeout(time)            # Suspend until entity arrives at process 1
      if i <=  5:
        print('Student %d arrives at time %.2f' % (i,self.env.now))
      self.env.process(self.process1(i))      # Execute first process
  def process1(self,i):
    '''Generic sim code for entity processing'''
    p = 1                                     # Label indicating process number
    arr_time = self.env.now                   # Record arrival time at process1
    time = 1+19*np.random.rand()              # Define process time for entity i
    yield self.env.timeout(time)              # Suspend until entity finishes at process 1
    if i <=  5:
      print('Student %d works out for %.2f minutes & is done at time %.2f' % (i,time,self.env.now))
    data.append([i,p,arr_time,time,self.env.now])  # Record process1 statistics for entity i
    if np.random.rand() <= 0.8:               # Check if entity goes to 2nd process
      self.env.process(self.process2(i))      # Execute second process
  def process2(self,i):
    '''Generic sim code for entity processing'''
    p = 2                                     # Label indicating process number
    arr_time = self.env.now                   # Record arrival time at process1
    time = max(0,np.random.normal(20,5))      # Define process time for entity i
    yield self.env.timeout(time)              # Suspend until entity finishes at process 1
    if i <=  5:
      print('Student %d uses locker room for %.2f minutes & leaves gym at time %.2f' % (i,time,self.env.now))
    data.append([i,p,arr_time,time,self.env.now])  # Record process2 statistics for entity i

data = []
env=simpy.Environment()
simShell(env,data)
env.run(until=200)                            # Simulation runs for desired number of periods
student_df = pd.DataFrame(data)               # Store the data in a convenient object (panda dataframe)
student_df.columns = ['student_no','process_no','arr_time',
                     'service_time','end_time'] # Add column labels
student_df                                    # Display the data frame

Student 1 arrives at time 5.57
Student 2 arrives at time 7.93
Student 3 arrives at time 12.52
Student 1 works out for 7.63 minutes & is done at time 13.21
Student 4 arrives at time 15.56
Student 5 arrives at time 15.56
Student 4 works out for 1.54 minutes & is done at time 17.11
Student 3 works out for 4.64 minutes & is done at time 17.16
Student 2 works out for 18.28 minutes & is done at time 26.22
Student 5 works out for 18.98 minutes & is done at time 34.55
Student 4 uses locker room for 25.38 minutes & leaves gym at time 42.49
Student 3 uses locker room for 26.13 minutes & leaves gym at time 43.29
Student 5 uses locker room for 20.87 minutes & leaves gym at time 55.42


,student_no,process_no,arr_time,service_time,end_time
0,1,1,5.573973,7.631713,13.205686
1,4,1,15.561617,1.544549,17.106166
2,3,1,12.523844,4.636834,17.160678
3,6,1,18.411900,2.362931,20.774832
4,7,1,20.055640,4.370362,24.426002
...,...,...,...,...,...
101,58,1,174.062245,16.942606,191.004850
102,59,2,179.870334,15.492440,195.362774
103,60,1,178.425426,18.513571,196.938997
104,62,1,183.954530,13.462061,197.416592


# 2. Question

So let us change the model by assuming that there are finite capacities for both the workout zone (the first activity) and the locker room (the second acitivity). Specifically, the workout zone can accommodate at most 20 students and the locker room can accommodate at most 10 students. Let us also assume that the workout time is uniformly distributed between 30 to 60 minutes.

Your goal is to **revise the code above by incorporating the the capacity constraints of these resources and run the simulation for 100,000 time units to  answer the following question:**
- What are the average wait times for the workout zone and the locker room?
- What are the utilization rates for these two resources?

# 3. Implementation

Let us first implement the model as folows. Note that in this case, we initialize the object with two more arguments `workout` and `lr`. They represent the resources. Also, in the previous simulation, students never wait but now to keep track of the wait time, we need to record the start time of each activity.

In [11]:
class simShell:
  def __init__(self,env,data,workout,lr):
    '''Generic sim code for single arrival process with sequential arrivals'''
    self.env = env                            # Define env as attribute of class 'simShell' (included as an argument to class methods via 'self')
    self.data = data                          # Define data as attribute of class 'simShell' to collect simulation data
    self.workout = workout                    # Define the resource, workout
    self.lr = lr                              # Define the reource, locker room
    self.env.process(self.arrival())          # Begin arrival process
  def arrival(self):
    i = 0
    while True:
      i = i + 1                               # Arrival number
      time = np.random.exponential(3)         # Define interarrival time for entity i
      yield self.env.timeout(time)            # Suspend until entity arrives at process 1
      if i <=  5:
        print('Student %d arrives at time %.2f' % (i,self.env.now))
      self.env.process(self.process1(i))      # Execute first process
  def process1(self,i):
    '''Generic sim code for entity processing'''
    p = 1                                     # Label indicating process number
    arr_time = self.env.now                   # Record arrival time at process1
    with self.workout.request() as req:       # Entity requests 1 unit of the resource
      yield req
      start_time = self.env.now               # Define start time. This is the time point when the student is done waiting and starts the service
      ###### Add your code here (two lines of code) ######
      time = np.random.uniform(30,60)                                        # Define process time for entity i, which is uniformly distributed between 30 to 60
      yield self.env.timeout(time)                                        # Suspend until entity finishes at process 1
      ###### end your code here ######
    if i <=  5:
      print('Student %d works out for %.2f minutes & is done at %.2f' % (i,time,env.now))
    data.append([i,p,arr_time,start_time,time,env.now])  # Record process1 statistics for entity i
    if np.random.rand() <= 0.8:                 # Check if entity goes to 2nd process
      self.env.process(self.process2(i))        # Execute second process
  def process2(self,i):
    '''Generic sim code for entity processing'''
      ###### Add your code here ######
      # Hint: Your code here should be very similar to the content in `process1`.
      # Keep in mind that you need to define `start_time`, which is the the time point when the student is done waiting and starts using the locker room.
      # You also need to append that to the list `data`
    p = 2
    arr_time = self.env.now
    with self.lr.request() as req:
      yield req
      start_time = self.env.now
      time = max(0, np.random.normal(20,5))
      yield self.env.timeout(time)

    if i<=5:
      print(f'Student {i} uses locker room for{time: .2f} minutes and leaves the gym at {self.env.now:.2f}')

    self.data.append([i, p, arr_time, start_time, time, self.env.now])
      ###### end your code here ######

# Run the simulation for 100,000 time units
data = []
env = simpy.Environment()
workout = simpy.Resource(env, capacity=20) #workout zone max 20 students
lr = simpy.Resource(env, capacity=10) #locker room max 10 students
model = simShell(env, data, workout, lr)
env.run(until=100000)

# Convert to DataFrame
df = pd.DataFrame(data, columns=['student_no', 'process_no', 'arr_time', 'start_time', 'service_time', 'end_time'])

# Calculate wait times
df['wait_time'] = df['start_time'] - df['arr_time']

# Compute average wait times
avg_wait_workout = df[df['process_no']==1]['wait_time'].mean()
avg_wait_locker = df[df['process_no']==2]['wait_time'].mean()

print(f'average wait time for workout is: {avg_wait_workout:.2f} minutes' )
print(f'average wait time for locker room is: {avg_wait_locker:.2f} minutes' )

Student 1 arrives at time 8.30
Student 2 arrives at time 13.86
Student 3 arrives at time 16.57
Student 4 arrives at time 18.16
Student 5 arrives at time 23.05
Student 2 works out for 39.13 minutes & is done at 52.99
Student 4 works out for 45.25 minutes & is done at 63.41
Student 1 works out for 57.29 minutes & is done at 65.59
Student 2 uses locker room for 17.15 minutes and leaves the gym at 70.13
Student 5 works out for 47.24 minutes & is done at 70.29
Student 3 works out for 57.21 minutes & is done at 73.77
Student 4 uses locker room for 19.38 minutes and leaves the gym at 82.78
Student 5 uses locker room for 15.57 minutes and leaves the gym at 85.86
Student 1 uses locker room for 21.11 minutes and leaves the gym at 86.70
Student 3 uses locker room for 19.14 minutes and leaves the gym at 92.92
average wait time for workout is: 0.85 minutes
average wait time for locker room is: 0.08 minutes


Next, we notice that to compute the utilization rate, we need to monitor the resources with funciton wrappers. So now we patch the resources and run the simulation. A key point is that now we have two different resources `workout` and `lr`. We patch them separately.

In [12]:
###### Add your code here: Implement the two functions `patch_resource` and `monitor` ######
# Hint: It should be the same as what we had in the class
def patch_resource(resource,pre=None,post=None):
  def get_wrapper(func):
    # Generate a wrapper for request/release
    @wraps(func)

    def wrapper(*args,**kwargs):
      if pre:
        pre(resource)
      # Perform operation as specified in 'func', ie, 'getattr' in this example
      # Between pre and post to yield difference in timing of data collection
      ret = func(*args,**kwargs)
      if post:
        post(resource)
      return ret

    return wrapper

  # Replace the original operations with our wrapper
  for name in ['request','release']:
    if hasattr(resource,name):
      setattr(resource,name,get_wrapper(getattr(resource,name)))

def monitor(r_data,resource):
  '''This is our monitoring callback'''
  item = (
          resource._env.now,                # The current simulation time
          resource.count,                   # The number of users of resource
          len(resource.queue),              # The number of queued processes
         )
  r_data.append(item)

###### end your code here ######
total_time = 100000
data = []
r_data_workout = []
r_data_lr = []
env = simpy.Environment()
workout = simpy.Resource(env,capacity=20)          # Dfine the resource
lr = simpy.Resource(env,capacity=10)               # Dfine the resource
simShell(env,data,workout,lr)

#################### Start: Process function patch commands ####################
### Patch the resource `workout`
monitor_workout = partial(monitor,r_data_workout)  # Bind the function `mointor` with `r_data_workout`
patch_resource(workout,pre=monitor_workout)        # Patch the resource

###### Add your code here: Patch the resource `lr` (two lines of code)######
# Hint: the code should be very similar to aboove. You need to apply `partial` on `monitor` to bind it with `r_data_lr` to obtain a new function `mointor_lr`.
# Then, you patch the resource `lr` with `mointor_lr`.
monitor_lr = partial(monitor,r_data_lr)                                                   # Bind the function `mointor` with `r_data_lr`
patch_resource(lr,pre=monitor_lr)                                                   # Patch the resource
###### end your code here ######
###################### End: Process function patch commands ####################

env.run(until=total_time)                          # Simulation runs for desired number of periods

Student 1 arrives at time 3.64
Student 2 arrives at time 4.89
Student 3 arrives at time 10.29
Student 4 arrives at time 10.76
Student 5 arrives at time 11.67
Student 2 works out for 33.28 minutes & is done at 38.17
Student 4 works out for 32.98 minutes & is done at 43.73
Student 3 works out for 47.29 minutes & is done at 57.58
Student 4 uses locker room for 15.95 minutes and leaves the gym at 59.68
Student 1 works out for 56.63 minutes & is done at 60.27
Student 2 uses locker room for 25.91 minutes and leaves the gym at 64.08
Student 5 works out for 57.43 minutes & is done at 69.10
Student 3 uses locker room for 18.72 minutes and leaves the gym at 76.30
Student 5 uses locker room for 27.01 minutes and leaves the gym at 96.10


Then, we compute the wait times.

In [14]:
### Compute the wait time for the work out zone
student_df = pd.DataFrame(data, columns = ['student_no','process_no','arr_time','start_time', 'service_time','end_time'])
                                                                                # Store the data in a convenient object (panda dataframe)
student_df_p1 = student_df[student_df["process_no"] == 1].copy()                # Slice data and obtain the data regarding the first process (workout zone)
waittime_wo = (student_df_p1["start_time"] - student_df_p1["arr_time"]).mean()  # Compute the average wait time for the workout zone
###### Add your code here: Compute the wait time for locker room and store it to variable `waittime_lr` (two lines of code) ######
# Hint: The code should be similar to the last two lines above.
student_df_p2 = student_df[student_df["process_no"] == 2].copy()
waittime_lr = (student_df_p2["start_time"] - student_df_p2["arr_time"]).mean()                    # Slice data and obtain the data regarding the first process (locker room zone)
                                                                                # Compute the average wait time for the locker room zone
###### end your code here ######
print("The average wait times for the work out zone and the locker room are %.2f mins and %.2f mins respectively." %(waittime_wo, waittime_lr))

The average wait times for the work out zone and the locker room are 1.01 mins and 0.09 mins respectively.


The last step is to compute the utilization rates. In general, the utililization rate can be written as
$
\frac{
\sum_{k}\text{The amount of time that the system has $k$ customers} \times k
}{\text{total time}\times \text{capacity}}.
$

In [21]:
### Compute the utilization rate for the workout zone
r_df_wo = pd.DataFrame(r_data_workout, columns = ['end_time', 'count','queue']) # Store the data in a convenient object (panda dataframe)
r_df_wo["start_time"] = np.concatenate(([0],np.array(r_df_wo.iloc[:-1, :]["end_time"])))
                                                                                # Add a new column for storing the start time
util_wo = ((r_df_wo['end_time'] - r_df_wo['start_time'])*r_df_wo['count']).sum()/(total_time*20)
                                                                                # Compute the utilization rate for the workout zone

###### Add your code here: Compute the utilization rate for the locker room and store it in `util_lr` (three lines of code) ######
# Hint: The code should be similar to the last three lines above.
r_df_lr = pd.DataFrame(r_data_lr, columns=['end_time','count','queue'])                                  # Store the data in a convenient object (panda dataframe)
r_df_lr['start_time']  =  np.concatenate(([0], np.array(r_df_lr.iloc[:-1,:]['end_time'])))                    # Add a new column for storing the start time
                                                                                # Compute the utilization rate for the locker room
util_lr = ((r_df_lr['end_time'] - r_df_lr['start_time'])*r_df_lr['count']).sum()/(total_time*20)
###### end your code here ######
print("The utilization rates of the workout zone and the locker room are", round(100*util_wo,2),"and",round(100*util_lr,2))

The utilization rates of the workout zone and the locker room are 75.32 and 26.69
